In [184]:
# Data Science
import re
import json
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
from pandasgui import show
import matplotlib.pyplot as plt

# General
import os
import sys
import time
import math
import random
from datetime import date
import warnings
current_date = date.today()
warnings.filterwarnings("ignore")

In [484]:
total = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\summary_stats_7_14.csv")

In [532]:
matches = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\MASTER_7_14.csv")

In [533]:
matches

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,...,merged_regex_match,padded_merged_row_location,padded_merged_regex_sent,padded_char_count,note_char_count,PCPID,EncounterEpicProviderID,pcp_flag,SpecialtyDSC,padded_regex_match
0,0,0,0,0,0.0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,...,[' cerebral'],"(5457, 6251)",ked at HMS in the department of psychobiology ...,794,19217.0,NaN,1521494.0,0,NaN,['cerebral']
1,1,1,1,1,1.0,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,...,[' memory'],"(3248, 4044)",or lymphadenopathy. Chest: Clear to auscultati...,796,5850.0,NaN,1012865.0,0,NaN,['memory']
2,2,2,2,2,14.0,100000272,10040002098,Z6352398,3.242118e+09,2019-03-14 00:00:00.0000000,...,[' dementia'],"(8345, 9145)",ession alone in the meta-analysis. We discusse...,800,13298.0,1002888.0,1668416.0,0,NaN,['dementia']
3,3,3,3,3,15.0,100000272,10040002098,Z6352398,3.322836e+09,2020-12-04 00:00:00.0000000,...,"[' moca', ' cognition', ' memory']","[(930, 1152), (4778, 4987), (5004, 5349)]",------- ing tylenol really accept for this we...,803,8711.0,1002888.0,1746848.0,0,NaN,"['cognition', 'memory', 'moca']"
4,4,4,4,4,20.0,100000272,10040002098,Z6352398,3.325561e+09,2020-09-18 00:00:00.0000000,...,"[' cognition', ' memory']","[(5188, 5576), (5593, 5991)]",------- in is related to cancer. Reassuringly...,804,9145.0,1002888.0,1746848.0,0,NaN,"['cognition', 'memory']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279219,279219,60817,60817,277874,NaN,113705759,10127549029,Z16320629,3.285425e+09,2020-04-28 00:00:00.0000000,...,"[' memory', ' cognition', ' cerebral', ' cereb...","[(538, 547), (1626, 1635), (1844, 1853), (108,...","------- se ""she took a break"" and wants to dis...",1949,11314.0,1007065.0,1558180.0,0,NaN,"['cognition', 'memory', 'cerebellar', 'cerebral']"
279220,279220,60818,60818,277883,NaN,113705759,10127549029,Z16320629,3.305416e+09,2020-09-22 00:00:00.0000000,...,"[' memory', ' cognition', ' cerebral', ' cereb...","[(323, 332), (1411, 1420), (1629, 1638), (108,...","------- all of her medications, including asp...",1734,10113.0,1007065.0,1558180.0,0,NaN,"['cognition', 'memory', 'cerebellar', 'cerebral']"
279221,279221,60819,60819,277896,NaN,113705759,10127549029,Z16320629,3.327430e+09,2021-01-25 00:00:00.0000000,...,"[' cognition', ' cerebral', ' memory', ' cereb...","[(2816, 2824), (108, 117), (326, 335), (974, 9...","------- ? Patient, Chart, daughter Olga Mode o...",3009,12320.0,1007065.0,1558180.0,0,NaN,"['dementia', 'cognition', 'memory', 'mci', 'ce..."
279222,279222,60820,60820,277912,NaN,113705759,10127549029,Z16320629,3.349755e+09,2021-04-20 00:00:00.0000000,...,"[' cognition', ' cerebral', ' cognitive impair...","[(2103, 2111), (108, 117), (326, 335), (544, 5...","------- ? Patient, Chart, daughter Olga Mode o...",2835,11250.0,1007065.0,1558180.0,0,NaN,"['dementia', 'cognition', 'moca', 'cognitive i..."


In [238]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\EDA\Getting_Data\keywords.csv")

In [240]:
subset.append(pd.DataFrame(matches.iloc[64072]).T)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,...,merged_regex_match,padded_merged_row_location,padded_merged_regex_sent,padded_char_count,note_char_count,PCPID,EncounterEpicProviderID,pcp_flag,SpecialtyDSC,padded_regex_match
89365,89365,39178,48709,48709,208499.0,101266523,10052675518,Z8852902,3148474490.0,2017-04-07 00:00:00.0000000,...,[' memory'],"[(457, 1257)]",his toes rub on his shoe wear. Patient Active ...,800,6873.0,1000678.0,1009975.0,0,NaN,['memory']
19058,19058,19058,23473,23473,100737.0,100104972,1181531,Z6739331,3281171897.0,2020-02-19 00:00:00.0000000,...,"[' cognition', ' memory']","[(62, 320), (3187, 3447), (4327, 4585)]",------- -------------------------------------...,803,9518.0,1121398.0,1558429.0,0,NaN,"['cognition', 'memory']"
9272,9272,9272,11446,11446,48515.0,100053648,658881,Z6457920,3215400829.0,2018-08-29 00:00:00.0000000,...,[' cognition'],"(18789, 19584)",I&O reviewed in Epic. Physical Exam: General: ...,795,20353.0,1000601.0,NaN,0,NaN,['cognition']
27240,27240,27242,33591,33591,144461.0,100146857,10030226061,Z7055023,3260611884.0,2020-08-04 00:00:00.0000000,...,"[' memory', ' cognitive impairment']","[(1952, 2247), (3276, 3482), (8003, 8284)]",------- ter than 5 feet? Greater than Left Ea...,809,12098.0,1229796.0,1229796.0,1,NaN,"['memory', 'cognitive impairment']"
17764,17764,17764,21896,21896,93447.0,100098036,10044342664,Z6686916,3105116945.0,2016-06-23 00:00:00.0000000,...,[' cerebral'],"(5687, 6479)","al Status Change, Shortness of Breath, Intrave...",792,16161.0,1001357.0,1000407.0,0,NaN,['cerebral']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66754,66754,16558,20660,20660,88209.0,100593503,10071891732,Z10462744,3265378685.0,2019-08-27 00:00:00.0000000,...,[' cognition'],"[(0, 750)]",----------------------------------------------...,750,844.0,1008591.0,1109286.0,0,NaN,['cognition']
49109,49109,49118,60834,60834,263468.0,100354602,10058767194,Z8633877,3166425155.0,2017-08-07 00:00:00.0000000,...,"[' cognition', ' memory']","[(16010, 16401), (17059, 17453)]",------- . Do not drive while taking narcotic ...,803,18286.0,1010669.0,NaN,0,NaN,"['cognition', 'memory']"
3114,3114,3114,3830,3830,15466.0,100022696,2052421,Z6377397,3102101124.0,2015-12-16 00:00:00.0000000,...,[' cognition'],"[(9632, 10025), (23702, 24101)]",------- t was WBAT with bilateral upper extre...,810,24439.0,1000932.0,NaN,0,NaN,['cognition']
48766,48766,48775,60435,60435,262035.0,100351338,10023987752,Z8609090,3197451981.0,2018-04-13 00:00:00.0000000,...,"[' dementia', ' memory']","[(9808, 10180), (11224, 11638)]",------- ubstance Abuse History: No known fami...,804,13946.0,1005782.0,700149.0,0,NaN,"['dementia', 'memory']"


In [241]:
TOTAL_KEYWORD_MATCHES = list(total["Total_Count"].unique())[1]

In [293]:
diverse_df = pd.DataFrame(columns = list(matches.columns))

In [243]:
percent = 5000 / TOTAL_KEYWORD_MATCHES

int(math.ceil(24 * percent))

1

In [248]:
len(partial_subset)

12061

In [449]:
keywords = ['dementia', 'cognition', 'memory', 'mmse', 'moca', 'alzheimer', 'cognitive impairment', 'mci', 'cerebellar', 'neurocognitive', 'lewy', "pick's",  'corticobasal', 'cerebral', 'cerebrovascular', 'amnesia', 'ad', 'lbd']

In [466]:
def diversity_sampling(SEQ_COUNT):
    # Constant declarations
    diverse_df = pd.DataFrame(columns = list(matches.columns))

    for word in (keywords):
        # getting subset of dataframe that contains sequence with keyword match
        contains = matches['padded_regex_match'].str.contains(str(word))
        filtered = matches[contains]
        subset = pd.DataFrame(columns = list(matches.columns))  
        # calculating percent that will be taken from each set of keyword matches
        percent = SEQ_COUNT / TOTAL_KEYWORD_MATCHES
        # getting number of matches to take from subset of keyword matches
        take = int(math.ceil(len(filtered) * percent))
        
        if (take == 1):
            take = 2
            
        print("Word:", word, "Take: ", take , "EMPI: ", len(filtered["EMPI"]), "Unique EMPI: ", len(filtered["EMPI"].unique()))

        if (take <= len(filtered["EMPI"].unique())):
            # priortizing getting sequences from unique patients if size of take permits
            filtered = filtered.drop_duplicates(subset = "EMPI", keep = "first")
            
            unique_empi = diverse_df["EMPI"].unique().tolist()
            
            filtered["in_diverse_df"] = np.where(filtered["EMPI"].isin(unique_empi), 'Y', 'N')
            new_additions = filtered[filtered["in_diverse_df"] == 'N']
            new_additions = pd.DataFrame(new_additions, columns = list(matches.columns))
            
#             print("New Additions: ", len(new_additions))
            idx = 0
            if take <= len(new_additions):
                idx = take
            else:
                idx = new_additions - 1
            new_additions = new_additions.iloc[:idx]
            subset = subset.append(new_additions)
            print("Subset After New Additions: ", len(subset), "Subset Unique EMPI: ", len(subset["EMPI"].unique().tolist()))
                    
            take -= len(new_additions)
            
            if (take > 0):
                filtered.drop(new_additions.index, axis = 0, inplace = True)

                # generating random index numbers for subset
                for _ in range(take):
                    idx = random.sample(range(0, len(filtered)), take)
                    subset = subset.append(pd.DataFrame(filtered.iloc[idx]).T)

                print("Subset: ", len(subset))
        else:
            # getting as many unique patient records as possible even if take > len(filtered["EMPI"].unique())
            partial_subset = filtered.drop_duplicates(subset = "EMPI", keep = "first")
            print("PS: ", len(partial_subset))
            subset.append(partial_subset)
            take -= len(partial_subset)
            filtered.drop(partial_subset.index, axis = 0, inplace = True)

            # generating random index numbers for subset
            for _ in range(take):
                idx = random.sample(range(0, len(filtered)), take)
                subset = subset.append(pd.DataFrame(matches.iloc[idx]).T)
                #print(len(subset))  

        diverse_df = diverse_df.append(subset)
        print("Unique EMPI: ", len(diverse_df["EMPI"].unique()), "Len: ", len(diverse_df), "\n")
    return diverse_df

In [467]:
diverse_df = diversity_sampling(5000)

Word: dementia Take:  593 EMPI:  51034 Unique EMPI:  5163
Subset After New Additions:  593 Subset Unique EMPI:  593
Unique EMPI:  593 Len:  593 

Word: cognition Take:  1018 EMPI:  87655 Unique EMPI:  13455
Subset After New Additions:  1018 Subset Unique EMPI:  1018
Unique EMPI:  1611 Len:  1611 

Word: memory Take:  1268 EMPI:  109218 Unique EMPI:  12061
Subset After New Additions:  1268 Subset Unique EMPI:  1268
Unique EMPI:  2879 Len:  2879 

Word: mmse Take:  25 EMPI:  2134 Unique EMPI:  796
Subset After New Additions:  25 Subset Unique EMPI:  25
Unique EMPI:  2904 Len:  2904 

Word: moca Take:  114 EMPI:  9767 Unique EMPI:  1963
Subset After New Additions:  114 Subset Unique EMPI:  114
Unique EMPI:  3018 Len:  3018 

Word: alzheimer Take:  239 EMPI:  20581 Unique EMPI:  2556
Subset After New Additions:  239 Subset Unique EMPI:  239
Unique EMPI:  3257 Len:  3257 

Word: cognitive impairment Take:  236 EMPI:  20267 Unique EMPI:  2957
Subset After New Additions:  236 Subset Unique EM

In [468]:
diverse_df = diverse_df.reset_index()
diverse_df

,index,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,EMPI,MRN,PatientID,PatientEncounterID,...,merged_regex_match,padded_merged_row_location,padded_merged_regex_sent,padded_char_count,note_char_count,PCPID,EncounterEpicProviderID,pcp_flag,SpecialtyDSC,padded_regex_match
0,2,2,2,2,2,14.0,100000272,10040002098,Z6352398,3.242118e+09,...,[' dementia'],"(8345, 9145)",ession alone in the meta-analysis. We discusse...,800,13298.0,1002888.0,1668416.0,0,NaN,['dementia']
1,21,21,21,27,27,97.0,100000894,200766,Z6352922,3.181499e+09,...,"[' moca', ' dementia', ' memory']","[(1007, 1396), (5123, 5521)]","------- sis over her R bra strap, which was i...",805,8302.0,1004526.0,1701875.0,0,NaN,"['dementia', 'memory', 'moca']"
2,35,35,35,41,41,123.0,100001150,10027033512,Z6353136,3.018789e+09,...,"[' dementia', ' cognitive impairment']","[(293, 571), (1185, 1405), (7169, 7447)]",------- l obstruction ASSOCIATED DIAGNOSES Sm...,803,15036.0,1006199.0,1011716.0,0,NaN,"['dementia', 'cognitive impairment']"
3,75,75,75,89,89,248.0,100001521,10040011735,Z6353461,3.172504e+09,...,"[' dementia', ' cognition']","[(3255, 3648), (4365, 4757)]",------- (156 lb) 09/19/17 71.4 kg (157 lb 6....,803,5360.0,1000333.0,1000333.0,1,NaN,"['dementia', 'cognition']"
4,139,139,139,166,166,538.0,100001873,10040014499,Z6353764,3.165983e+09,...,[' dementia'],"(7578, 8374)",ut answer. Optho notes from recent outpatient ...,796,9382.0,1004526.0,1701875.0,0,NaN,['dementia']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5004,102789,102789,6929,8453,8453,38620.0,101886663,15974728,Z6566087,3.019130e+09,...,[' cognition'],"[(10966, 11765)]",er lisinopril and dyazide on discharge. Please...,799,18940.0,NaN,1009238.0,0,NaN,"['cognition', 'ad']"
5005,102993,102993,7133,8740,8740,39894.0,101892417,1205814,Z6593900,3.317829e+09,...,"[' dementia', ' memory']","[(0, 355), (456, 849)]",----------------------------------------------...,757,1501.0,1121398.0,1775091.0,0,NaN,"['dementia', 'memory', 'ad']"
5006,110531,110531,14685,18195,18195,81874.0,102238452,10033260075,Z8801390,3.186800e+09,...,[' cognition'],"[(624, 1421)]","ding w/ close S and grab bar, HHSH Dressing: U...",797,6040.0,1000254.0,NaN,0,NaN,"['cognition', 'lbd']"
5007,114632,114632,18786,23257,23257,103376.0,102424707,10022003700,Z9965570,3.306047e+09,...,[' memory'],"[(3219, 4013)]",dressing with LHAE. Pt able to improve perform...,794,6652.0,1001669.0,NaN,0,NaN,"['memory', 'lbd']"


In [469]:
len(diverse_df["PatientID"].unique())

5009

In [422]:
diverse_df.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\Diversity_Sampling\5000_sample_7_19.csv")

In [454]:
x=matches["padded_regex_match"].value_counts()
print(x)

['cognition']                                                                                                                  49873
['memory']                                                                                                                     46655
['dementia']                                                                                                                   20914
['cerebral']                                                                                                                   20911
['cerebrovascular']                                                                                                            18085
                                                                                                                               ...  
['cognition', 'memory', 'moca', 'alzheimer', 'cognitive impairment', 'cerebellar', 'cerebral']                                     1
['moca', 'cognitive impairment', 'mci', 'cerebral']                  

In [528]:
# defining summary stats df
cols =  ['dementia', 'cognition', 'memory', 'mmse', 'moca', 'alzheimer', 'cognitive impairment', 'mci', 'cerebellar', 'neurocognitive', 'lewy', "pick's",  'corticobasal', 'cerebral', 'cerebrovascular', 'amnesia', 'ad', 'lbd']

summary_stats_seq  = pd.DataFrame(pd.np.empty((len(diverse_df), len(cols) + 3)) * pd.np.nan) 
summary_stats_seq.columns = ['EMPI', 'EncounterID', 'NoteID'] + list(cols)
summary_stats_seq["EMPI"] = diverse_df["EMPI"].to_list()
summary_stats_seq["EncounterID"] = diverse_df["PatientEncounterID"].to_list()
summary_stats_seq["NoteID"]  = diverse_df["NoteID"].to_list()

In [530]:
for i in tqdm(range(len(d))):
    match_tuples = []
    keyword_list = []
    
    keyword_list = list(eval(diverse_df['padded_regex_match'][i]))
    for z in range(len(keyword_list)):
        match_tuples.append((keyword_list[z], 0))

    freq = {"dementia":0
        ,"cognition":0
        ,"cognition":0
        ,"memory":0
        ,"mmse":0
        ,"moca":0
        ,"alzheimer":0
        ,'cognitive impairment':0
        ,"mci":0
        ,"cerebellar":0
        ,"neurocognitive":0
        ,"lewy":0
        ,"pick's":0
        ,"corticobasal":0
        ,"cerebral":0
        ,"cerebrovascular":0
        ,"amnesia":0
        ,"ad": 0
        ,"lbd": 0
    }

    for k, v in match_tuples:   
        if (k != ''):
            if (k == 'cognitive impairmentcognitiveimpairment' or k == 'cognitiveimpairmentcognitive impairment'):
                k = 'cognitive impairment'
                freq.update({str(k.lower()):int(freq[k.lower()] + 1)})
            else:
                freq.update({str(k.lower()):int(freq[k.lower()] + 1)})
            
    #print(freq.items())
    #print(summary_stats["dementia"])
    for k, v in freq.items():
        #print(str(k.lower()))
        summary_stats_seq[str(k.lower())][i] = int(v)

  0%|                                                                                         | 0/5009 [00:00<?, ?it/s]


KeyError: '['

In [425]:
summary_stats_seq

,EMPI,EncounterID,NoteID,dementia,cognition,memory,mmse,moca,alzheimer,cognitive impairment,...,cerebellar,neurocognitive,lewy,pick's,corticobasal,cerebral,cerebrovascular,amnesia,ad,lbd
0,100000272,3.242118e+09,2370049295,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100000894,3.181499e+09,1916263837,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100001150,3.018789e+09,267681869,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100001521,3.172504e+09,1682366001,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100001873,3.165983e+09,1853759390,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5004,101886663,3.019130e+09,267890258,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5005,101892417,3.317829e+09,4335598103,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5006,102238452,3.186800e+09,1846710034,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5007,102424707,3.306047e+09,4028154201,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [185]:
show(summary_stats_seq)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI...
PandasGUI INFO — pandasgui.gui — Opening PandasGUI...
PandasGUI INFO — pandasgui.gui — Opening PandasGUI...


In [426]:
summary_stats_seq.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\Diversity_Sampling\5000_seqs_7_19.csv")

In [427]:
slat_excel = pd.read_excel(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\Diversity_Sampling\5000_SLAT_import_7_19.xlsx")

In [428]:
slat_excel

,PatientID,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match
0,Z6352398,10040002098,100000272,3242118020,Progress Notes,2370049295,2338373237,2019-03-14,----------------------------------------------...,"""ession alone in the meta-analysis. We discuss...","""['dementia']"""
1,Z6352922,200766,100000894,3181499472,Progress Notes,1916263837,1871981644,2018-03-28,----------------------------------------------...,""" ------- sis over her R bra strap, which was ...","""['dementia', 'memory', 'moca']"""
2,Z6353136,10027033512,100001150,3018789480,Discharge Summary,267681869,255525481,2015-01-09,----------------------------------------------...,""" ------- l obstruction ASSOCIATED DIAGNOSES S...","""['dementia', 'cognitive impairment']"""
3,Z6353461,10040011735,100001521,3172504002,Progress Notes,1682366001,1631850265,2017-10-03,----------------------------------------------...,""" ------- (156 lb) 09/19/17 71.4 kg (157 lb 6...","""['dementia', 'cognition']"""
4,Z6353764,10040014499,100001873,3165982636,Progress Notes,1853759390,1807702879,2018-02-07,----------------------------------------------...,"""ut answer. Optho notes from recent outpatient...","""['dementia']"""
...,...,...,...,...,...,...,...,...,...,...,...
5004,Z6566087,15974728,101886663,3019130054,Discharge Summary,267890258,255731280,2014-08-07,----------------------------------------------...,"""er lisinopril and dyazide on discharge. Pleas...","""['cognition', 'ad']"""
5005,Z6593900,1205814,101892417,3317828810,NaN,4335598103,4321848674,2020-07-29,----------------------------------------------...,"""---------------------------------------------...","""['dementia', 'memory', 'ad']"""
5006,Z8801390,10033260075,102238452,3186799957,Progress Notes,1846710034,1800448290,2018-01-22,----------------------------------------------...,"""ding w/ close S and grab bar, HHSH Dressing: ...","""['cognition', 'lbd']"""
5007,Z9965570,10022003700,102424707,3306047330,Progress Notes,4028154201,4011889394,2020-05-08,----------------------------------------------...,"""dressing with LHAE. Pt able to improve perfor...","""['memory', 'lbd']"""


In [430]:
slat_excel.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\Diversity_Sampling\5000_SLAT_import_7_20.csv", index = False)

In [431]:
## JSON format is incorrect, so rerunning regex matches
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\EDA\Getting_Data\keywords.csv")

In [432]:
k = regex["REGEX"].to_list()
c = regex["CASE"].to_list()
p_list = []

for i in range(len(k)):
    if (c[i] == 0):
        p_list.append(re.compile(k[i][5:], re.IGNORECASE))
    elif (c[i] == 1):
        p_list.append(re.compile(k[i]))
print(p_list)

[re.compile('\\bdementia\\b', re.IGNORECASE), re.compile('\\bcognition\\b', re.IGNORECASE), re.compile('\\bmemory\\b', re.IGNORECASE), re.compile('\\bMMSE\\b', re.IGNORECASE), re.compile('\\bMOCA\\b', re.IGNORECASE), re.compile('\\bAlzheimer\\b', re.IGNORECASE), re.compile('\\bcognitive\\s*impairment\\b', re.IGNORECASE), re.compile('(?-i:)\\bMCI\\b'), re.compile('\\bcerebellar\\b', re.IGNORECASE), re.compile('\\bNeurocognitive\\b', re.IGNORECASE), re.compile('\\blewy\\b', re.IGNORECASE), re.compile("\\bpick's\\b", re.IGNORECASE), re.compile('\\bCorticobasal\\b', re.IGNORECASE), re.compile('\\bcerebral\\b', re.IGNORECASE), re.compile('\\bcerebrovascular\\b', re.IGNORECASE), re.compile('\\bamnesia\\b', re.IGNORECASE), re.compile('(?-i:)\\bAD\\b'), re.compile('(?-i:)\\bLBD\\b')]


In [433]:
string = (slat_excel["regex_sent"][5]) + " mCi."

print(string)

for p in p_list:
    for match in re.finditer(p, string):
        s = match.start()
        e = match.end()
        print ('String match "%s" at %d:%d' % (string[s:e], s, e))

"hat would be more consistent. Has been alert/oriented for the past two days with apparent agitation/sundowning overnight. Son provided additional information that pt has a history of delusions that proceed any recent hospital admission, describing that people are breaking into his home and that he has been assaulted. Also describes auditory hallucination of someone singing. May be related to dementia (Lewey body although tends to be more associated with visual hallucinations vs. vascular) vs. Tinnitus. Dx: - F/u HIV - B12, TSH wnl Tx: - CTM sx of infection, low threshold for abx - Bisacodyl suppository PRN and senna bowel regimen for regular BMs - Appreciate geri recs: OOB at least TID and with meals, consolidate metoprolol, start finasteride - F/u with geriatrics regarding new informati" mCi.
String match "dementia" at 396:404


In [440]:
locations = []

for i in tqdm(range(len(slat_excel['regex_sent']))):
    r = str(slat_excel['regex_sent'][i])
    l = []
    for p in p_list:
        for match in re.finditer(p, r):
            l.append(match.group().lower())
            l = list(set(l))
    #print(l)
    locations.append(l)
        
slat_excel["regex_match"] = locations

100%|████████████████████████████████████████████████████████████████████████████| 5009/5009 [00:02<00:00, 1992.02it/s]


In [442]:
slat_excel['regex_match'] = slat_excel['regex_match'].apply(json.dumps)

In [443]:
slat_excel

,PatientID,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match
0,Z6352398,10040002098,100000272,3242118020,Progress Notes,2370049295,2338373237,2019-03-14,----------------------------------------------...,"""ession alone in the meta-analysis. We discuss...","[""dementia""]"
1,Z6352922,200766,100000894,3181499472,Progress Notes,1916263837,1871981644,2018-03-28,----------------------------------------------...,""" ------- sis over her R bra strap, which was ...","[""dementia"", ""memory"", ""moca""]"
2,Z6353136,10027033512,100001150,3018789480,Discharge Summary,267681869,255525481,2015-01-09,----------------------------------------------...,""" ------- l obstruction ASSOCIATED DIAGNOSES S...","[""cognitive impairment"", ""dementia""]"
3,Z6353461,10040011735,100001521,3172504002,Progress Notes,1682366001,1631850265,2017-10-03,----------------------------------------------...,""" ------- (156 lb) 09/19/17 71.4 kg (157 lb 6...","[""cognition"", ""dementia""]"
4,Z6353764,10040014499,100001873,3165982636,Progress Notes,1853759390,1807702879,2018-02-07,----------------------------------------------...,"""ut answer. Optho notes from recent outpatient...","[""dementia""]"
...,...,...,...,...,...,...,...,...,...,...,...
5004,Z6566087,15974728,101886663,3019130054,Discharge Summary,267890258,255731280,2014-08-07,----------------------------------------------...,"""er lisinopril and dyazide on discharge. Pleas...","[""ad"", ""cognition""]"
5005,Z6593900,1205814,101892417,3317828810,NaN,4335598103,4321848674,2020-07-29,----------------------------------------------...,"""---------------------------------------------...","[""dementia"", ""memory"", ""ad""]"
5006,Z8801390,10033260075,102238452,3186799957,Progress Notes,1846710034,1800448290,2018-01-22,----------------------------------------------...,"""ding w/ close S and grab bar, HHSH Dressing: ...","[""cognition"", ""lbd""]"
5007,Z9965570,10022003700,102424707,3306047330,Progress Notes,4028154201,4011889394,2020-05-08,----------------------------------------------...,"""dressing with LHAE. Pt able to improve perfor...","[""memory"", ""lbd""]"


In [444]:
slat_excel["regex_match"].unique()

array(['["dementia"]', '["dementia", "memory", "moca"]',
       '["cognitive impairment", "dementia"]',
       '["cognition", "dementia"]', '["dementia", "memory", "cerebral"]',
       '["dementia", "cerebral"]',
       '["cerebrovascular", "dementia", "cerebral", "cognitive impairment", "alzheimer"]',
       '["cerebellar", "dementia"]', '["dementia", "neurocognitive"]',
       '["dementia", "moca"]', '["memory", "dementia"]',
       '["dementia", "alzheimer"]', '["dementia", "memory"]',
       '["memory", "dementia", "moca"]',
       '["cognitive impairment", "dementia", "memory"]',
       '["dementia", "alzheimer", "cerebrovascular"]',
       '["dementia", "cerebrovascular"]',
       '["cerebellar", "dementia", "memory"]',
       '["cognitive impairment", "dementia", "cerebral"]',
       '["memory", "cognition", "dementia"]',
       '["dementia", "cognition", "memory", "cerebrovascular"]',
       '["memory", "cerebellar", "dementia"]',
       '["ad", "dementia", "cerebral"]', '["dem

In [445]:
slat_excel.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\nlp_annotator-dev\app\load_data\5000_SLAT_import_7_20_v4.csv", index = False)

In [228]:
slat_excel["ContactDTS"] = pd.to_datetime(slat_excel["ContactDTS"], format='%Y-%m-%d')

In [230]:
example = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\nlp_annotator-dev\app\load_data\regex_dataset.csv")

In [232]:
example.isna().sum()

PatientID                 0
MRN                     275
EMPI                    138
PatientEncounterID      670
InpatientNoteTypeDSC    931
NoteID                    0
NoteCSNID                 0
ContactDTS                0
NoteTXT                   0
regex_sent                0
regex_match               0
dtype: int64

In [233]:
slat_excel.isna().sum()

PatientID                0
MRN                      0
EMPI                     0
PatientEncounterID       0
InpatientNoteTypeDSC    41
NoteID                   0
NoteCSNID                0
ContactDTS               0
NoteTXT                  0
regex_sent               0
regex_match              0
dtype: int64

In [473]:
d = pd.read_excel(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\Diversity_Sampling\5000_SLAT_import_7_19.xlsx")

In [474]:
d

,PatientID,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match
0,Z6352398,10040002098,100000272,3242118020,Progress Notes,2370049295,2338373237,2019-03-14,----------------------------------------------...,"""ession alone in the meta-analysis. We discuss...","""['dementia']"""
1,Z6352922,200766,100000894,3181499472,Progress Notes,1916263837,1871981644,2018-03-28,----------------------------------------------...,""" ------- sis over her R bra strap, which was ...","""['dementia', 'memory', 'moca']"""
2,Z6353136,10027033512,100001150,3018789480,Discharge Summary,267681869,255525481,2015-01-09,----------------------------------------------...,""" ------- l obstruction ASSOCIATED DIAGNOSES S...","""['dementia', 'cognitive impairment']"""
3,Z6353461,10040011735,100001521,3172504002,Progress Notes,1682366001,1631850265,2017-10-03,----------------------------------------------...,""" ------- (156 lb) 09/19/17 71.4 kg (157 lb 6...","""['dementia', 'cognition']"""
4,Z6353764,10040014499,100001873,3165982636,Progress Notes,1853759390,1807702879,2018-02-07,----------------------------------------------...,"""ut answer. Optho notes from recent outpatient...","""['dementia']"""
...,...,...,...,...,...,...,...,...,...,...,...
5004,Z6566087,15974728,101886663,3019130054,Discharge Summary,267890258,255731280,2014-08-07,----------------------------------------------...,"""er lisinopril and dyazide on discharge. Pleas...","""['cognition', 'ad']"""
5005,Z6593900,1205814,101892417,3317828810,NaN,4335598103,4321848674,2020-07-29,----------------------------------------------...,"""---------------------------------------------...","""['dementia', 'memory', 'ad']"""
5006,Z8801390,10033260075,102238452,3186799957,Progress Notes,1846710034,1800448290,2018-01-22,----------------------------------------------...,"""ding w/ close S and grab bar, HHSH Dressing: ...","""['cognition', 'lbd']"""
5007,Z9965570,10022003700,102424707,3306047330,Progress Notes,4028154201,4011889394,2020-05-08,----------------------------------------------...,"""dressing with LHAE. Pt able to improve perfor...","""['memory', 'lbd']"""


In [490]:
d.columns

Index(['PatientID', 'MRN', 'EMPI', 'PatientEncounterID',
       'InpatientNoteTypeDSC', 'NoteID', 'NoteCSNID', 'ContactDTS', 'NoteTXT',
       'regex_sent', 'regex_match'],
      dtype='object')

In [537]:
matches.drop(d.index, axis = 0, inplace = True)

In [538]:
matches = matches[['PatientID', 'MRN', 'EMPI', 'PatientEncounterID',
       'InpatientNoteTypeDSC', 'NoteID', 'NoteCSNID', 'ContactDTS', 'NoteTXT',
       'regex_sent', 'regex_match']]

In [539]:
matches

,PatientID,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match
5009,Z6398203,2579886,100030990,3.284218e+09,Progress Notes,5786184612,5.783659e+09,2021-05-10 00:00:00.0000000,----------------------------------------------...,"'on and coordination, normal speech. Psych: Be...",['cognition']
5010,Z6398203,2579886,100030990,3.340324e+09,Progress Notes,4962623319,5.111476e+09,2020-12-04 00:00:00.0000000,----------------------------------------------...,' normal coordination and gait. Psych: Alert a...,['cognition']
5011,Z6398203,2579886,100030990,3.340837e+09,Progress Notes,5881291037,5.892214e+09,2021-05-25 00:00:00.0000000,----------------------------------------------...,' other lesions. Normal turgor. Psych: Alert a...,['cognition']
5012,Z6398203,2579886,100030990,3.347308e+09,Letter,5452283829,5.447225e+09,2021-03-12 00:00:00.0000000,----------------------------------------------...,"'on and coordination, normal speech. Psych: Be...",['cognition']
5013,Z6398230,1577089,100030999,3.181851e+09,Progress Notes,1794875152,1.747131e+09,2017-12-19 00:00:00.0000000,----------------------------------------------...,'tter with markedly diminished right sided ret...,['memory']
...,...,...,...,...,...,...,...,...,...,...,...
279219,Z16320629,10127549029,113705759,3.285425e+09,Progress Notes,3960511043,3.943607e+09,2020-04-28 00:00:00.0000000,----------------------------------------------...,"""'she took a break' and wants to discuss with ...","[""memory""]"
279220,Z16320629,10127549029,113705759,3.305416e+09,Progress Notes,4585859437,4.574134e+09,2020-09-22 00:00:00.0000000,----------------------------------------------...,"""all of her medications, including aspirin. De...","[""memory""]"
279221,Z16320629,10127549029,113705759,3.327430e+09,Progress Notes,5203067949,5.224024e+09,2021-01-25 00:00:00.0000000,----------------------------------------------...,"""Patient, Chart, daughter Olga Mode of contact...","[""cognition""]"
279222,Z16320629,10127549029,113705759,3.349755e+09,Progress Notes,5673786544,5.670392e+09,2021-04-20 00:00:00.0000000,----------------------------------------------...,"""Patient, Chart, daughter Olga Mode of contact...","[""cognition""]"


In [515]:
matches.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\Diversity_Sampling\not_processed_matches.csv", index = False)

## Executing SLAT Pipeline Preprocessing

In [557]:

df_not_padded = matches #pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\Diversity_Sampling\not_processed_matches.csv") 
df_not_padded

,PatientID,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match
5009,Z6398203,2579886,100030990,3.284218e+09,Progress Notes,5786184612,5.783659e+09,2021-05-10,----------------------------------------------...,"'on and coordination, normal speech. Psych: Be...",['cognition']
5010,Z6398203,2579886,100030990,3.340324e+09,Progress Notes,4962623319,5.111476e+09,2020-12-04,----------------------------------------------...,' normal coordination and gait. Psych: Alert a...,['cognition']
5011,Z6398203,2579886,100030990,3.340837e+09,Progress Notes,5881291037,5.892214e+09,2021-05-25,----------------------------------------------...,' other lesions. Normal turgor. Psych: Alert a...,['cognition']
5012,Z6398203,2579886,100030990,3.347308e+09,Letter,5452283829,5.447225e+09,2021-03-12,----------------------------------------------...,"'on and coordination, normal speech. Psych: Be...",['cognition']
5013,Z6398230,1577089,100030999,3.181851e+09,Progress Notes,1794875152,1.747131e+09,2017-12-19,----------------------------------------------...,'tter with markedly diminished right sided ret...,['memory']
...,...,...,...,...,...,...,...,...,...,...,...
279219,Z16320629,10127549029,113705759,3.285425e+09,Progress Notes,3960511043,3.943607e+09,2020-04-28,----------------------------------------------...,"""'she took a break' and wants to discuss with ...","[""memory""]"
279220,Z16320629,10127549029,113705759,3.305416e+09,Progress Notes,4585859437,4.574134e+09,2020-09-22,----------------------------------------------...,"""all of her medications, including aspirin. De...","[""memory""]"
279221,Z16320629,10127549029,113705759,3.327430e+09,Progress Notes,5203067949,5.224024e+09,2021-01-25,----------------------------------------------...,"""Patient, Chart, daughter Olga Mode of contact...","[""cognition""]"
279222,Z16320629,10127549029,113705759,3.349755e+09,Progress Notes,5673786544,5.670392e+09,2021-04-20,----------------------------------------------...,"""Patient, Chart, daughter Olga Mode of contact...","[""cognition""]"


In [565]:
merged = df_not_padded
merged["ContactDTS"] = pd.to_datetime(merged["ContactDTS"]).dt.strftime('%Y-%m-%d')

In [509]:
# merged['regex_sent'] = merged['regex_sent'].apply(json.dumps)
# merged['regex_match'] = merged['regex_match'].apply(json.dumps)

In [547]:
test = '["moca", "cerebellar"]'

In [548]:
test = test.replace('"', "'")

In [549]:
test

"['moca', 'cerebellar']"

In [553]:
print(merged["regex_match"].tolist()[0][0])

[


In [566]:
merged["regex_match"] = merged["regex_match"].apply(lambda x : x.replace('"', "'"))

In [569]:
merged['regex_sent'] = merged['regex_sent'].apply(json.dumps)
merged['regex_match'] = merged['regex_match'].apply(json.dumps)

In [570]:
merged

,PatientID,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match
5009,Z6398203,2579886,100030990,3.284218e+09,Progress Notes,5786184612,5.783659e+09,2021-05-10,----------------------------------------------...,"""'on and coordination, normal speech. Psych: B...","""['cognition']"""
5010,Z6398203,2579886,100030990,3.340324e+09,Progress Notes,4962623319,5.111476e+09,2020-12-04,----------------------------------------------...,"""' normal coordination and gait. Psych: Alert ...","""['cognition']"""
5011,Z6398203,2579886,100030990,3.340837e+09,Progress Notes,5881291037,5.892214e+09,2021-05-25,----------------------------------------------...,"""' other lesions. Normal turgor. Psych: Alert ...","""['cognition']"""
5012,Z6398203,2579886,100030990,3.347308e+09,Letter,5452283829,5.447225e+09,2021-03-12,----------------------------------------------...,"""'on and coordination, normal speech. Psych: B...","""['cognition']"""
5013,Z6398230,1577089,100030999,3.181851e+09,Progress Notes,1794875152,1.747131e+09,2017-12-19,----------------------------------------------...,"""'tter with markedly diminished right sided re...","""['memory']"""
...,...,...,...,...,...,...,...,...,...,...,...
279219,Z16320629,10127549029,113705759,3.285425e+09,Progress Notes,3960511043,3.943607e+09,2020-04-28,----------------------------------------------...,"""\""'she took a break' and wants to discuss wit...","""['memory']"""
279220,Z16320629,10127549029,113705759,3.305416e+09,Progress Notes,4585859437,4.574134e+09,2020-09-22,----------------------------------------------...,"""\""all of her medications, including aspirin. ...","""['memory']"""
279221,Z16320629,10127549029,113705759,3.327430e+09,Progress Notes,5203067949,5.224024e+09,2021-01-25,----------------------------------------------...,"""\""Patient, Chart, daughter Olga Mode of conta...","""['cognition']"""
279222,Z16320629,10127549029,113705759,3.349755e+09,Progress Notes,5673786544,5.670392e+09,2021-04-20,----------------------------------------------...,"""\""Patient, Chart, daughter Olga Mode of conta...","""['cognition']"""


In [571]:
production = pd.concat([d, matches])

In [572]:
production

,PatientID,MRN,EMPI,PatientEncounterID,InpatientNoteTypeDSC,NoteID,NoteCSNID,ContactDTS,NoteTXT,regex_sent,regex_match
0,Z6352398,10040002098,100000272,3.242118e+09,Progress Notes,2370049295,2.338373e+09,2019-03-14,----------------------------------------------...,"""ession alone in the meta-analysis. We discuss...","""['dementia']"""
1,Z6352922,200766,100000894,3.181499e+09,Progress Notes,1916263837,1.871982e+09,2018-03-28,----------------------------------------------...,""" ------- sis over her R bra strap, which was ...","""['dementia', 'memory', 'moca']"""
2,Z6353136,10027033512,100001150,3.018789e+09,Discharge Summary,267681869,2.555255e+08,2015-01-09,----------------------------------------------...,""" ------- l obstruction ASSOCIATED DIAGNOSES S...","""['dementia', 'cognitive impairment']"""
3,Z6353461,10040011735,100001521,3.172504e+09,Progress Notes,1682366001,1.631850e+09,2017-10-03,----------------------------------------------...,""" ------- (156 lb) 09/19/17 71.4 kg (157 lb 6...","""['dementia', 'cognition']"""
4,Z6353764,10040014499,100001873,3.165983e+09,Progress Notes,1853759390,1.807703e+09,2018-02-07,----------------------------------------------...,"""ut answer. Optho notes from recent outpatient...","""['dementia']"""
...,...,...,...,...,...,...,...,...,...,...,...
279219,Z16320629,10127549029,113705759,3.285425e+09,Progress Notes,3960511043,3.943607e+09,2020-04-28,----------------------------------------------...,"""\""'she took a break' and wants to discuss wit...","""['memory']"""
279220,Z16320629,10127549029,113705759,3.305416e+09,Progress Notes,4585859437,4.574134e+09,2020-09-22,----------------------------------------------...,"""\""all of her medications, including aspirin. ...","""['memory']"""
279221,Z16320629,10127549029,113705759,3.327430e+09,Progress Notes,5203067949,5.224024e+09,2021-01-25,----------------------------------------------...,"""\""Patient, Chart, daughter Olga Mode of conta...","""['cognition']"""
279222,Z16320629,10127549029,113705759,3.349755e+09,Progress Notes,5673786544,5.670392e+09,2021-04-20,----------------------------------------------...,"""\""Patient, Chart, daughter Olga Mode of conta...","""['cognition']"""


In [573]:
production.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\Diversity_Sampling\SLAT_production_7_22.csv", index = False)

In [574]:
production["ContactDTS"].unique()

array(['2019-03-14', '2018-03-28', '2015-01-09', ..., '2007-11-24',
       '2006-10-25', '2004-02-19'], dtype=object)